In [ ]:
from IPython.display import display, Javascript, HTML
js = '''
    element.textContent = "Hello IPython.display!";
    '''

display(Javascript(js))   

In [ ]:
from notebookjs import execute_js

In [ ]:
hello_js = """
function hello(selector, data){
    document.querySelector(selector).textContent=data.text;
}
"""

In [ ]:
execute_js(hello_js, "hello", {"text": "Hello NotebookJS"})

In [ ]:
with open("./hello_comps.js", "r") as f:
    comps_lib = f.read()

In [ ]:
execute_js(comps_lib, "comps", {"test":"cookies"})

In [ ]:
execute_js(comps_lib, "comps", {"test":"signin","username":"psylwester","password":"Password123"})

In [ ]:
execute_js(comps_lib, "comps", {"test":"window"})